In [20]:

"""
This script defines the overall exercise for ATIAM structure course

 - Use this as a baseline script
 - You are authorized to define other files for functions
 - Write a (small) report document (PDF) explaining your approach
 - All your files should be packed in a zip file named
     [ATIAM][FpA2020]FirstName_LastName.zip

@author: esling
"""

# Basic set of imports (here you can see if everything passes)
from music21 import converter
import numpy as np
import math
from needleman import needleman_affine
from needleman import needleman_simple
import string
import pickle
import os
# Define mode (keep it on False, this is just for my generative part)
DEV_MODE = False

if DEV_MODE:
    # Define MIDI extension
    midExt = ['mid', 'midi', 'MID', 'MIDI']
    # Root directory
    root = '/Users/esling/Coding/learning/orchestral-db/data'
    database = {}
    composers = []
    composers_tracks = {}
    tracks = []
    # List composers first
    for item in os.listdir(root):
        if os.path.isdir(os.path.join(root, item)):
            composers.append(item)
    print('Found ' + str(len(composers)) + ' composers.')
    prev_letter = ''
    # now parse tracks
    for comp in sorted(composers):
        # Print advance
        if (comp[0] != prev_letter):
            prev_letter = comp[0]
            print('   - Composers starting with ' + prev_letter)
        # Check each sub-folder
        for item in os.listdir(os.path.join(root, comp)):
            cur_path = os.path.join(os.path.join(root, comp), item)
            if os.path.isdir(cur_path):
                for files in os.listdir(cur_path):
                    if (os.path.splitext(files)[1][1:] in midExt):
                        tracks.append(item)
                        if comp in composers_tracks.keys():
                            composers_tracks[comp].append(item)
                        else:
                            composers_tracks[comp] = [item]
    print('Found ' + str(len(tracks)) + ' tracks.')
    midi_database = {'composers': composers,
                     'composers_tracks': composers_tracks}
    pickle.dump(midi_database, open("atiam-fpa.pkl", "wb"))
else:
    midi_database = pickle.load(open("atiam-fpa.pkl", "rb"))
    composers = midi_database['composers']
    composers_tracks = midi_database['composers_tracks']

# Here an example: print all composers with more than 10 tracks
for composer, tracks in sorted(composers_tracks.items()):
    if (len(tracks) >= 10):
        print(composer + ' : ' + str(len(tracks)) + ' tracks.')


Abel, Carl Friedrich : 27 tracks.
Aboyan, Gayk : 554 tracks.
Abt, Franz : 38 tracks.
Adam, Adolphe : 11 tracks.
Adson, John : 71 tracks.
Agincour, François d' : 15 tracks.
Agrell, Johan : 84 tracks.
Agricola, Alexander : 12 tracks.
Aguado, Dionisio : 28 tracks.
Ahle, Johann Rudolf : 18 tracks.
Aiblinger, Johann Kaspar : 18 tracks.
Aichinger, Gregor : 12 tracks.
Alain, Jehan : 13 tracks.
Albeniz, Isaac : 49 tracks.
Albert, Heinrich : 10 tracks.
Albinoni, Tomaso : 170 tracks.
Albrechtsberger, Johann Georg : 89 tracks.
Aleotti, Vittoria : 18 tracks.
Alexandra, Liana : 12 tracks.
Alink, Bert : 22 tracks.
Alkan, Charles-Valentin : 30 tracks.
Allegri, Lorenzo : 12 tracks.
Allison, Richard : 177 tracks.
Alsen, Wulf Dieter : 38 tracks.
Altenburg, Johann Ernst : 28 tracks.
Altenburg, Michael : 26 tracks.
Alıcıoğlu, Şafak : 75 tracks.
Ammer, Manfred : 114 tracks.
Anderson, Leigh : 10 tracks.
André, Johann Anton : 11 tracks.
Anglebert : 67 tracks.
Anonymous : 27 tracks.
Anonymus, .... : 899 t

In [21]:
#
from typing import Any, Callable, Union
import pandas as pd
import util as ut

unit_testing = True
unit_testing_log_level = 2

### PART 1 - Exploring a track collections (text dictionnaries) and playing with MIDI

In this part, we will start easy by looking at a collection of tracks.
The set of classical music pieces is provided in the _atiam-fpa.pkl_ file, which
is already loaded at this point of the script and contain two structures
    - composers         = Array of all composers in the database
    - composers_tracks  = Hashtable of tracks for a given composer


### Q-1.1 Re-implement one of the array sorting algorithm seen in class
### either bubble sort or quicksort
### +1 point bonus for quicksort

In [22]:
def pip_sort(arr: np.ndarray, comp: Callable[[Any, Any], int] = None, axis: int = -1, comp_is_rec: bool=True, pivot_idx: int = 0) -> None:
    """[summary]
    In-place quick-sorting of input 1D-array, according to input comparison function.
    Args:
        array (np.ndarray): input nD-array to be sorted. 
        comp (Callable[[Any, Any], int]): If None, will sort in increasing order. 
            If comp_is_rec, will sort scalar_type of input array.
            If not, will sort (n-1-axis)D-sub-arrays of input array.
        axis (int, optional): [description]. Defaults to -1 for last axis.
        comp_is_rec(bool): if True, the sub-arrays will be sorted recursively lexicographically.
            Used if array is multi-dimensional, and axis is not -1.
    """
    def swap_el(a: np.ndarray, i: int, j: int) -> None:
        # see: https://stackoverflow.com/a/47951813
        a[[i, j]] = a[[j, i]]

    def quicksort_rec(a:np.ndarray, start_idx:int, stop_idx:int, pivot_idx:int)->None:
        """Recursive step
        Args:
            a (np.ndarray): The (n-1-axis)D-sub-array of input array to be sorted.
            start_idx ([type]): Start index of sub-array to be sorted
            stop_idx ([type]): Stop index of sub-array to be sorted (inclusive)
            pivot_idx ([type]): Index of pivot, should be in [start_idx, stop_idx]
        """
        # print(f"sort {start_idx}:{stop_idx}, pivot={pivot_idx}]")
        # print(f"a: {a}")
        sub_arr_len = stop_idx-start_idx+1
        if sub_arr_len <= 1:
            # nothing to do
            return
        elif sub_arr_len == 2:
            c = comp(a[start_idx], a[stop_idx])
            if c>0:
                swap_el(a, start_idx, stop_idx)
                return
        # if pivot is not the starting element, swap em
        if pivot_idx!=start_idx:
            swap_el(a, start_idx, pivot_idx)
        # sort_start_idx should now point to the first element to be processed
        # after pivot_idx.
        sort_start_idx, pivot_idx = pivot_idx+1, start_idx
        # Using pivot to partition the array
        left_idx, right_idx = sort_start_idx, stop_idx
        # for it in range(sort_start_idx, stop_idx+1):
        while left_idx <= right_idx:
            while left_idx <= right_idx and comp(a[left_idx], a[pivot_idx]) <= 0:
                left_idx += 1
            while right_idx >= left_idx and comp(a[right_idx], a[pivot_idx]) >= 0:
                right_idx -= 1
            if left_idx < right_idx:
                # swap the two
                swap_el(a, left_idx, right_idx)
        # Finally, swap right with pivot
        swap_el(a, pivot_idx, right_idx)
        pivot_idx, right_idx = right_idx, pivot_idx
        # Recrusive step
        quicksort_rec(a, start_idx, pivot_idx-1, start_idx) # left
        quicksort_rec(a, pivot_idx+1, stop_idx, pivot_idx+1) # right
    # set default comparison: increasing order
    if comp is None:
        def comp(a, b):
            if a > b:
                return 1
            elif a < b:
                return -1
            else:
                return 0
    if axis == None:
        raise AttributeError("Use out-of-place version for sorting flattened version of the array")
    if comp_is_rec:
        np.apply_along_axis(lambda _a: quicksort_rec(_a, 0, np.shape(_a)[0]-1, pivot_idx), axis=axis, arr=arr)
    else:
        Ni = np.shape(arr)[:axis]
        for ii in np.ndindex(Ni):
            quicksort_rec(arr[ii + np.s_[:, ]], 0, np.shape(arr)[axis]-1, pivot_idx)


def pip_sorted(arr: np.ndarray, comp: Callable[[Any, Any], int] = None, axis: Union[int, None] = -1, comp_is_rec: bool=True, pivot_idx: int = 0) -> None:
    """[summary]
    Out-of-place quick-sorting of input 1D-array, according to input predicate.
    See quick_sort for information.
    """
    arr_sorted = None
    if axis is None:
        arr_sorted = np.flatten(arr)
        axis = 0
    else:
        arr_sorted = np.copy(arr)
    pip_sort(arr_sorted, comp=comp, axis=axis, comp_is_rec=comp_is_rec, pivot_idx=pivot_idx)
    return arr_sorted


### Q-1.2 Use your own algorithm to sort the collection of composers by decreasing number of tracks

In [23]:
################
# YOUR CODE HERE
################
## Some unit testing
if unit_testing:
    rng = np.random.default_rng()
    for i in range(100):
        shape=rng.integers(1, 5, size=(3,))
        dim = rng.integers(1, 4)
        shape=tuple(rng.integers(1, 20, size=dim))
        n = np.multiply.reduce(shape)
        axis=rng.integers(0, dim)
        if unit_testing_log_level <= 1:
            print(f"shape={shape}, axis={axis}")
        arr = rng.integers(0, n, size=n)
        rng.shuffle(arr)
        arr = arr.reshape(shape)
        arr_sorted_truth = np.sort(arr, axis=axis)
        arr_sorted = pip_sorted(arr, axis=axis)
        if unit_testing_log_level <= 2 and np.linalg.norm(arr_sorted-arr_sorted_truth)>0.001:
            ut.log_error(arr_sorted_truth, arr_sorted)

In [35]:

composers = np.array(composers)
# see: https://stackoverflow.com/q/15579649
# and: https://stackoverflow.com/a/43187340
tracks = np.array([(comp, composers_tracks[comp] if comp in composers_tracks else []) for comp in composers], dtype=object)
## Some examples of the content of these structures
pip_sort(tracks, comp_is_rec=False, comp=lambda a,b: len(a[1]) - len(b[1]), axis=0)
composers = tracks[:][0]

In [32]:
for i in range(composers.shape[0]-1, 0, -1):
    print(f"Composer: {composers[i][0]}, #tracks={len(tracks[i][1])}")

Composer: M, #tracks=0
Composer: O, #tracks=0
Composer: M, #tracks=0
Composer: R, #tracks=0
Composer: F, #tracks=0
Composer: P, #tracks=0
Composer: S, #tracks=1
Composer: F, #tracks=0
Composer: V, #tracks=0
Composer: Z, #tracks=0
Composer: S, #tracks=0
Composer: K, #tracks=0
Composer: B, #tracks=0
Composer: H, #tracks=0
Composer: P, #tracks=0
Composer: C, #tracks=0
Composer: K, #tracks=0
Composer: G, #tracks=0
Composer: B, #tracks=0
Composer: L, #tracks=0
Composer: H, #tracks=0
Composer: S, #tracks=106
Composer: B, #tracks=0
Composer: V, #tracks=0
Composer: S, #tracks=3
Composer: B, #tracks=0
Composer: S, #tracks=0
Composer: S, #tracks=18
Composer: C, #tracks=0
Composer: B, #tracks=0
Composer: S, #tracks=0
Composer: F, #tracks=0
Composer: L, #tracks=1
Composer: O, #tracks=0
Composer: M, #tracks=0
Composer: S, #tracks=0
Composer: J, #tracks=0
Composer: D, #tracks=0
Composer: G, #tracks=0
Composer: C, #tracks=0
Composer: T, #tracks=0
Composer: O, #tracks=0
Composer: N, #tracks=0
Composer